In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
import requests
import os

spark = SparkSession.builder.appName("EIA Electricity Data").getOrCreate()

api_key = 'NPqxwCdaHco8Z5wWQxZCYcREzjU5lEXfvyT7uFka'

url = f"https://api.eia.gov/v2/electricity/electric-power-operational-data/data/?frequency=monthly&data[0]=ash-content&data[1]=consumption-for-eg&data[2]=consumption-for-eg-btu&data[3]=consumption-uto&data[4]=consumption-uto-btu&data[5]=cost&data[6]=cost-per-btu&data[7]=generation&data[8]=heat-content&data[9]=receipts&data[10]=receipts-btu&data[11]=stocks&data[12]=sulfur-content&data[13]=total-consumption&data[14]=total-consumption-btu&facets[location][]=TX&start=2015-01&end=2024-08&sort[0][column]=period&sort[0][direction]=asc&offset=0&length=5000&api_key={api_key}"

output_dir = "data"
os.makedirs(output_dir, exist_ok=True)

response = requests.get(url)

if response.status_code == 200:
    json_file_path = os.path.join(output_dir, "electricity_data.json")
    with open(json_file_path, "w") as json_file:
        json_file.write(response.text)
    print("Electricity data downloaded successfully.")

    df = spark.read.json(json_file_path)
else:
    print(f"Failed to fetch data: {response.status_code} - {response.text}")

# explode the response data to flatten the structure
exploded_df = df.select(
    "ExcelAddInVersion",
    "apiVersion",
    explode(df.response.data).alias("response_data")
)

# selects relevant fields from the exploded DataFrame
final_df = exploded_df.select(
    "ExcelAddInVersion",
    "apiVersion",
    "response_data.*"  # this will bring all fields from the response_data struct
)

print("Final DataFrame:")
final_df.show(truncate=False)
final_df.printSchema()



Electricity data downloaded successfully.
Final DataFrame:
+-----------------+----------+-----------+-----------------+------------------+----------------------+----------------------------+------------------------+---------------+-------------------+-------------------------+-----------------------+----+------------+-----------------------+--------------------------+----------------------------------------+----------+-----------+----------------------+------------+----------------------+--------+-------+---------+------------+------------------+-----------------------+---------------------------+--------+----------------+------+-----------------------+--------------+--------------------+-----------------+---------------------+---------------------------+-----------------------+
|ExcelAddInVersion|apiVersion|ash-content|ash-content-units|consumption-for-eg|consumption-for-eg-btu|consumption-for-eg-btu-units|consumption-for-eg-units|consumption-uto|consumption-uto-btu|consumption-uto-bt

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType

# Assuming final_df is already created and contains your data

# Drop rows with any null values
final_df = final_df.dropna()

# List of columns that need to be converted to numeric types
numeric_columns = [
    'ash-content',
    'consumption-for-eg',
    'consumption-for-eg-btu',
    'consumption-uto',
    'consumption-uto-btu',
    'cost',
    'cost-per-btu',
    'generation',
    'heat-content',
    'receipts',
    'stocks',
    'sulfur-content',
    'total-consumption',
    'total-consumption-btu'
]

for column in numeric_columns:
    final_df = final_df.withColumn(column, final_df[column].cast(DoubleType()))
final_df.show(truncate=False)
final_df.printSchema()



+-----------------+----------+-----------+-----------------+------------------+----------------------+----------------------------+------------------------+---------------+-------------------+-------------------------+---------------------+------+------------+-----------------------+----------------------+--------------------------+----------+----------+----------------------+------------+------------------+--------+-------+--------+------------+------------------+-------------------+---------------------------+--------+----------------+--------+-------------------+--------------+--------------------+-----------------+---------------------+---------------------------+-----------------------+
|ExcelAddInVersion|apiVersion|ash-content|ash-content-units|consumption-for-eg|consumption-for-eg-btu|consumption-for-eg-btu-units|consumption-for-eg-units|consumption-uto|consumption-uto-btu|consumption-uto-btu-units|consumption-uto-units|cost  |cost-per-btu|cost-per-btu-units     |cost-units     